In [6]:
import pandas as pd
import numpy as np
import os
import ast


In [10]:
# Creamos una lista
rows_items = []
# Abrimos el archivo JSON para LECTURA
with open('../data/australian_users_items.json', 'r', encoding='UTF-8') as items:
    # Iteramos a través de las líneas del archivo para obtener cada expresión de python
    for line in items.readlines():
               rows_items.append(ast.literal_eval(line))

In [11]:
#Se crea un frame a partir de los datos iterados
aus_items = pd.DataFrame(rows_items)
aus_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [12]:
#Se elimina la columna user_url que no sirve para nada
aus_items = aus_items.drop(columns=['user_url'])
aus_items.head()

,user_id,items_count,steam_id,items
0,76561197970982479,277,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [13]:
#Contamos cuantos registros hay
aus_items.shape[0]

88310

In [14]:
# Lista vacia donde se van a guardar cada iteracion desanidad de reviews
desanidados_items = []

# Entra a cada elemento de users_items
for index, row in aus_items.iterrows():
    # Obtiene los valores de las columnas 'user_id', 'steam_id' y 'items' de la fila actual
    user_id = row['user_id']
    steam_id = row['steam_id']
    items = row['items']

    # Itera a través de la lista de ítems en la columna 'items'
    for i in items:
        # Crea un diccionario con los valores desanidados
        items_desanidados = {
            'user_id': user_id,
            'steam_id': steam_id,
            'item_id': i.get('item_id', ''), # Obtiene el valor de 'item_id'
            'item_name': i.get('item_name',''), # Obtener el valor de 'item_name'
            'playtime_forever': i.get('playtime_forever',''), # Obtener el valor de 'playtime_forever'
            'playtime_2weeks': i.get('playtime_2weeks','') # Obtener el valor de 'playtime_2weeks'
        }
        # Agrega el diccionario a la lista des_items
        desanidados_items.append(items_desanidados)

# Crea un nuevo DataFrame a partir de la lista des_items
users_items = pd.DataFrame(desanidados_items)

users_items.head()

,user_id,steam_id,item_id,item_name,playtime_forever,playtime_2weeks
0,76561197970982479,76561197970982479,10,Counter-Strike,6,0
1,76561197970982479,76561197970982479,20,Team Fortress Classic,0,0
2,76561197970982479,76561197970982479,30,Day of Defeat,7,0
3,76561197970982479,76561197970982479,40,Deathmatch Classic,0,0
4,76561197970982479,76561197970982479,50,Half-Life: Opposing Force,0,0


In [15]:
# Elimina la columna 'steam_id'
users_items = users_items.drop(columns='steam_id')


In [16]:
# Filtra el DataFrame users_items para mantener solo las filas donde 'playtime_forever' no sea igual a 0
users_items = users_items[users_items['playtime_forever'] != 0]

In [18]:
# Guarda el DataFrame users_items en un archivo Parquet
users_items.to_csv(os.path.join('../data/', 'users_items.csv'), index=False)